In [1]:
import torch
pip install torch -f https://download.pytorch.org/whl/torch_stable.html

C:\Users\MrSwi\anaconda3\envs\projects\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch.nn as nn

In [8]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [9]:
from transformers import BertModel

In [10]:
tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-mini-arabic")

In [11]:
import pandas as pd
import string
from nltk.corpus import stopwords

In [12]:
df = pd.read_csv("proccesed.csv")

In [15]:
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# Arabic stop words with nltk
stop_words = stopwords.words()

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def text_preprocessing(text):
    '''
    text is an arabic string input
    
    the preprocessed text is returned
    '''
    
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text

In [16]:
import emoji
import unicodedata
import re
import numpy as np
def preprocessing_for_bert(data, tokinizer, text_preprocessing_fn = text_preprocessing, MAX_LEN=10):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []
    
    # For every sentence...
    for i,sent in enumerate(data):
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing_fn(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            padding='max_length',        # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True,     # Return attention mask
            truncation = True 
            )
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))
    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [17]:
def get_test_loader(data, tokinizer):
    input_id, attention_masks = preprocessing_for_bert(data, tokinizer)

In [18]:
df.reset_index(inplace=True)

In [19]:
X = df.Tweet
y = df.Sentiment.replace({"pos": 1, "neg": 0})

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X

0                   الله مير يبقيك ، بالتوفيق يارب ساروه 💋
1                                    ملكة الراب العالمية 👑
2                                        عقبال كل مسرحية 😂
3        اخواتي واخواني اللي بيتخانقوا لايف وبينشروا ال...
4                                             حاضر عمتيي 😒
                               ...                        
56790                 اغنيه مكملناش #تامر_عاشور 💔 #بصوتي 🎶
56791    امس من جد جاهدت من قلب لدرجة فتحت الاجهزة الظه...
56792    طيب ادلع زوجتي شوي فيها شي ابله غاده . يعني تر...
56793    أكثر الأندية الأوروبية تتويجا بلقب الدوري المح...
56794              بموت مره حلوه كيف تتكلم كذا نينيينينيني
Name: Tweet, Length: 56795, dtype: object

In [22]:
X_train, X_val, y_train, y_val = train_test_split(X.values,y,test_size=0.1, random_state=42)

In [23]:
type(X_train)

numpy.ndarray

In [24]:
train_inputs, train_masks = preprocessing_for_bert(X_train, tokenizer)
val_inputs, val_masks = preprocessing_for_bert(X_val, tokenizer)

In [75]:
device = torch.device("cpu")

In [76]:
type(train_inputs)

torch.Tensor

In [77]:
train_masks.shape

torch.Size([51115, 10])

In [78]:
train_inputs.shape

torch.Size([51115, 10])

In [79]:
device

device(type='cpu')

In [80]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_labels = torch.tensor(y_train.values)
val_labels = torch.tensor(y_val.values)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 16

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [81]:
train_labels.shape

torch.Size([51115])

In [82]:
def get_test_loader(data, tokinizer):
    input_id, attention_masks = preprocessing_for_bert(data, tokinizer)
    X_test = torch.tensor(input_ids)
    test_data = TensorDataset(X_test, attention_masks)
    test_sampler = RandomSampler(test_data)

In [83]:
class BertModel(nn.Module):
    """
    BERT :) 
    """
    
    def __init__(self, freeze=False):
        """
        
        """
        super(BertModel, self).__init__()
        D_in = 32000 # bert_in ?
        H, D_out= 16000,2
        
        self.bert =  AutoModelForMaskedLM.from_pretrained("asafaya/bert-mini-arabic")
        
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )
        if freeze:
            for param in self.bert.parameters():
                param.requires_grad = False
                
    def forward(self, input_ids, attention_mask):
        
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        last_hidden_state_cls = outputs[0][:, 0, :]
        
        logits = self.classifier(last_hidden_state_cls)
        
        return logits

In [84]:
bert =  AutoModelForMaskedLM.from_pretrained("asafaya/bert-base-arabic")

In [85]:
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.optim import SparseAdam, Adam

In [86]:
def initialize_model(epochs=4):
    
    bert_model = BertModel(freeze=False)
    
    bert_model = bert_model
    
    optimizer = AdamW(params=list(bert_model.parameters()), 
                     lr=5e-5,
                     eps=1e-8)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    
    return bert_model, optimizer, scheduler

In [87]:
import random
import time
import torch
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

def train(model, train_loader, val_loader, epochs=4):
    
    print("START TRAINING...")
    
    for epoch in range(epochs+1):
        total_loss, batch_loss, batch_counts = 0.,0.,0.
        
        model.train()
        
        for step,batch in enumerate(train_loader):
            batch_counts+=1
            
            inputs_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
            
            model.zero_grad()
            
            logits = model(inputs_ids, attention_mask)
            
            train_loss = criterion(logits,labels)
            
            batch_loss += train_loss.item()
            total_loss += train_loss.item()
            
            train_loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            scheduler.step()
            
            model.eval()
            
            val_losses, val_accuracy = [],[]
            
            input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
        
            with torch.no_grad():
                logits = model(input_ids, attention_mask)
        for batch in val_loader:
            val_loss = criterion(logits, labels)
            val_losses.append(val_loss.item())
            preds = torch.argmax(logits, dim=1).flatten()
            accuracy = (preds == b_labels).cpu().numpy().mean() * 100
            val_accuracy.append(accuracy)
            val_loss = np.mean(val_loss)
            val_accuracy = np.mean(val_accuracy)
            
        if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                
            print("epoch: {} | step: {} | train_loss: {} | validation_loss".format(epoch, step, (batch_loss / batch_counts), val_loss, val_accuracy))
            batch_loss, batch_counts = 0.,0.
            
            
                
            

In [88]:
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)

In [89]:
CUDA_LAUNCH_BLOCKING=1

In [ ]:
train(bert_classifier, train_dataloader, val_dataloader, epochs=2)

START TRAINING...
